In [110]:
# dataset link:  https://www.kaggle.com/competitions/fake-news/data

In [2]:
import pandas as pd
import numpy as np
import re  #for searching in text
from nltk.corpus import stopwords            #stopwordsleri belirler and, a
from nltk.stem.porter import PorterStemmer   #kelime kökünü bırakır
from sklearn.feature_extraction.text import TfidfVectorizer   #kelimeleri sayısal vektöre cevirir
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [3]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hsyn_\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [10]:
train_dataset=pd.read_csv("train.csv")
train_dataset.shape
train_dataset.head()


,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [17]:
train_dataset["label"].value_counts()  #uniform dataset

1    10413
0    10387
Name: label, dtype: int64

In [20]:
train_dataset.isnull().sum()   #558 title missing, 1957 author missing

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [24]:
#replacing null values with empty string
train_dataset=train_dataset.fillna("")
train_dataset.isnull().sum()  #all the missing values gone

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [59]:
train_dataset["content"]=train_dataset["author"]+" "+train_dataset["title"]

In [60]:
train_dataset["content"]

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object

In [62]:
port_stem=PorterStemmer()

In [63]:
def stemming(content): #harf olmayanların yerine bosluk geldi
    stemmed_content=re.sub("[^a-zA-Z]",' ',content)  #sadece kelimeleri tutuyor sayıları atıyor, noktalama isaretlerini atiyor
    stemmed_content=stemmed_content.lower()  #tüm harfleri lowera cevirdi
    stemmed_content=stemmed_content.split()   #kelime dizini liste cevrildi
    stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words("english")]  
    stemmed_content=' '.join(stemmed_content)  #stopword olmayanlar köklerine indirildi
    return stemmed_content
    

In [64]:
train_dataset["content"]=train_dataset["content"].apply(stemming)

In [65]:
print(train_dataset["content"])  # buyuk harfler rakamlar noktalar gitti

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [75]:
X=train_dataset["content"].values
Y=train_dataset["label"].values
X,Y


(array(['darrel lucu hous dem aid even see comey letter jason chaffetz tweet',
        'daniel j flynn flynn hillari clinton big woman campu breitbart',
        'consortiumnew com truth might get fire', ...,
        'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time',
        'alex ansari nato russia hold parallel exercis balkan',
        'david swanson keep f aliv'], dtype=object),
 array([1, 0, 1, ..., 0, 1, 1], dtype=int64))

In [77]:
vectorizer=TfidfVectorizer()  #kelimeler sayısal vektore cevrildi. y zaten number old. icin yapmamıza gerek yok
vectorizer.fit(X)
X=vectorizer.transform(X)


<20800x17128 sparse matrix of type '<class 'numpy.float64'>'
	with 210687 stored elements in Compressed Sparse Row format>

In [80]:
print(X)

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  (2, 5389)	0.3866530551182615
  (2, 3103)	0.46097489583229645
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396507
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.174553480255222
  (20797, 9518)	0.295420

In [88]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=2)  #splitting datas

model=LogisticRegression()
model.fit(X_train,Y_train)
y_pred=model.predict(X_test)
print(y_pred)
print(y_pred.size)

[1 1 1 ... 1 1 1]
4160


In [89]:
         #calculating test accuracy
test_data_accuracy=accuracy_score(y_pred,Y_test)
test_data_accuracy

0.9774038461538461

In [108]:
#trying the system
X_new=X_test[250]
prediction=model.predict(X_new)

Y_test[250]

0

In [109]:
prediction

array([0], dtype=int64)